In [1]:
import cv2
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
import os
from PIL import Image, ImageFile
from torch.utils.data import Dataset
import torch
import torch.optim as optim
import torchvision.transforms as transsforms

from torchvision import transforms
from torch.optim import lr_scheduler

In [2]:
class PupilDataset(Dataset):

    def __init__(self, pt, transform=None):
        self.pt = pt
        self.transform = transform

    def __len__(self):
        return len(self.pt)

    def __getitem__(self, idx):
        img_name = self.pt[idx]    
        image = Image.open(img_name)
        label = torch.tensor(train_csv.loc[idx,[2,3]].values.astype(np.float16))
        if self.transform:
            image = self.transform(image)
        return image, label

In [3]:
train_list = np.array([os.path.join(d, x) for d, dirs, files in os.walk('./train/') for x in files if x.endswith('.jpg')])
test_list = np.array([os.path.join(d, x) for d, dirs, files in os.walk('./test/') for x in files if x.endswith('.jpg')])
val_list = np.array([os.path.join(d, x) for d, dirs, files in os.walk('./val/') for x in files if x.endswith('.jpg')])

In [4]:
train_tf = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

trainset     = PupilDataset(train_list, transform =train_tf)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=0)
valset       = PupilDataset(val_list, transform =train_tf)
val_loader   = torch.utils.data.DataLoader(valset, batch_size=8, shuffle=False, num_workers=0)

In [5]:
train_csv = pd.read_csv('./target50bypeople.csv', header = None)
clusters = train_csv[4].values

In [6]:
x = 640/224
y = 480 / 224

train_csv[2] = train_csv[2] / x
train_csv[3] = train_csv[3] / y 

In [7]:
num_classes = 2
lr          = 1e-3
IMG_SIZE    = 224

In [8]:
from efficientnet_pytorch import EfficientNet
from torch.optim import Adam, SGD, RMSprop
import torch.nn.init as init
import torch
import torch.nn as nn

model = EfficientNet.from_pretrained('efficientnet-b0') 
in_features = model._fc.in_features
model._fc = nn.Linear(in_features, num_classes)
model.cuda()

Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

In [10]:
def train_model(epoch):
    model.train() 
        
    avg_loss = 0.
    optimizer.zero_grad()
    for idx, (imgs, labels) in enumerate(train_loader):
        imgs_train, labels_train = imgs.cuda(), labels.float().cuda()
        output_train = model(imgs_train)
        loss = criterion(output_train,labels_train)
        loss.backward()
        optimizer.step() 
        optimizer.zero_grad() 
        avg_loss += loss.item() / len(train_loader)
        
    return avg_loss

def test_model():
    
    avg_val_loss = 0.
    model.eval()
    with torch.no_grad():
        for idx, (imgs, labels) in enumerate(val_loader):
            imgs_vaild, labels_vaild = imgs.cuda(), labels.float().cuda()
            output_test = model(imgs_vaild)
            avg_val_loss += criterion(output_test, labels_vaild).item() / len(val_loader)
        
    return avg_val_loss

In [ ]:
import time


best_avg_loss = 30000
n_epochs      = 30

for epoch in range(n_epochs):
    
    print('lr:', scheduler.get_lr()[0]) 
    start_time   = time.time()
    avg_loss     = train_model(epoch)
    avg_val_loss = test_model()
    elapsed_time = time.time() - start_time 
    print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t time={:.2f}s'.format(
        epoch + 1, n_epochs, avg_loss, avg_val_loss, elapsed_time))
    
    if avg_val_loss < best_avg_loss:
        best_avg_loss = avg_val_loss
        torch.save(model.state_dict(), './weight_best_b0_bypeople.pt')
    
    scheduler.step()



lr: 0.001
Epoch 1/30 	 loss=6574.1164 	 val_loss=2576.4379 	 time=118.18s
lr: 0.001
Epoch 2/30 	 loss=1304.7814 	 val_loss=2003.4360 	 time=111.22s
lr: 0.001
Epoch 3/30 	 loss=1174.7918 	 val_loss=1777.2143 	 time=113.68s
lr: 0.001
Epoch 4/30 	 loss=1121.4515 	 val_loss=2025.9915 	 time=115.21s
lr: 0.001
Epoch 5/30 	 loss=1115.3164 	 val_loss=1802.7469 	 time=114.91s
lr: 0.001
Epoch 6/30 	 loss=1079.5944 	 val_loss=2348.2728 	 time=113.16s
lr: 0.001
Epoch 7/30 	 loss=1063.2870 	 val_loss=1969.2615 	 time=112.05s
lr: 0.001
Epoch 8/30 	 loss=1035.9294 	 val_loss=1819.0432 	 time=112.44s
lr: 0.001
Epoch 9/30 	 loss=1029.2973 	 val_loss=1991.8999 	 time=111.91s
lr: 0.001
Epoch 10/30 	 loss=1012.0877 	 val_loss=2078.9749 	 time=111.87s
lr: 0.001
Epoch 11/30 	 loss=992.5815 	 val_loss=1693.3011 	 time=111.89s
lr: 0.001
Epoch 12/30 	 loss=981.4650 	 val_loss=1802.6657 	 time=112.45s
lr: 0.001
Epoch 13/30 	 loss=986.2594 	 val_loss=1786.8853 	 time=113.06s
lr: 0.001
Epoch 14/30 	 loss=946.6818